In [72]:

import torch
import os
import segmentation_models_pytorch as smp
import torchvision.transforms as transforms
import cv2
import numpy as np
from PIL import Image
import torch

In [73]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = smp.UnetPlusPlus('efficientnet-b4', classes=1).to(device)
model.load_state_dict(torch.load("gay_sex.pth"))

<All keys matched successfully>

In [74]:
folder = 'МЫШИПТУ'
video_name = 'porno.m4v'
video_path = os.path.join(folder, video_name)
model.eval()

size = (320, 544)
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

preprocess  = transforms.Compose([transforms.Resize(size), transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)])
output_path = f'segmented_video_{video_name}.mp4'
show = True

In [75]:
video_capture = cv2.VideoCapture(video_path)
if not video_capture.isOpened():
    print("Ошибка: Невозможно открыть видео.")
    exit()

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video_capture.get(cv2.CAP_PROP_FPS)
output_video = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

show_output = True  # Define show_output as True or False based on your requirement

while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break

    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    input_tensor = preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        result = model(input_tensor).cpu().numpy().squeeze()
    mask = (result > 0).astype(np.uint8)
    resized_mask = cv2.resize((mask * 255).astype(np.uint8), (frame_width, frame_height))

    overlay = np.zeros_like(frame)
    overlay[resized_mask == 255] = [255, 255, 255]

    output_video.write(overlay)

    if show_output:
        cv2.imshow('Segmented Video', overlay)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

video_capture.release()
output_video.release()
cv2.destroyAllWindows()
